# Definición de la Red Neuronal

In [28]:
import numpy as np
import random
import pickle

In [128]:
class ModeloB:
    
    def __init__(self, dim_in  = 2, h_layers = [3 , 5], dim_out = 2, lr = 0.02):
        np.random.seed(505) #Establecida para experimentación
        random.seed(505)
        self.dim_in  = dim_in # Dimensión de la entrada
        self.dim_h_layers = len(h_layers) # Número de capas ocultas
        self.dim_out = dim_out # Dimensión capa de salida

        self.lr = lr #Learning Reate

        self.layers = h_layers
        
        self.layers.insert(0,dim_in)
        self.layers.append(dim_out)

        self.ident = lambda x : x
        self.tanh = lambda x : np.tanh(x)
        self.soft_max = lambda x : np.exp(x - np.max(x))/np.exp(x - np.max(x)).sum(0, keepdims=True) #Cero porque son vectores columna

        self.weigths = list() 
        self.bias = list()
        self.Hs = list()

        for k in range(self.dim_h_layers + 1):
            self.weigths.append(np.random.rand(self.layers[k+1],self.layers[k])/np.sqrt(self.layers[k+1]))#Pa que converja rápido
            self.bias.append(np.ones((self.layers[k+1],1)))
            self.Hs.append(np.zeros((self.layers[k+1],1)))

    def _backward(self,y_idx,x_idx):
        y = np.zeros((self.sizeVocaculary,1))
        y[y_idx] = 1

        d_salida = self.Hs[-1] - y
        d_hidden =(1 - self.Hs[-2]**2) * np.dot(self.weigths[-1].T,d_salida)
        d_embedd =  np.dot(self.weigths[-2].T,d_hidden)

        #Actualización de pesos
        self.weigths[-1] -=  self.lr * np.outer(d_salida,self.Hs[-2])
        self.weigths[-2] -=  self.lr * np.outer(d_hidden,self.Hs[-3])
        self.weigths[-3][:,x_idx] -= self.lr * d_embedd.reshape(-1) 

        #Actualización de bias
        self.bias[-1] = self.bias[-1] - self.lr * d_salida
        self.bias[-2] = self.bias[-2] - self.lr * d_hidden

    def _fordward(self, x):
        #Obtiene el embedding
        self.Hs[0] = self.weigths[0][:,x].reshape((100,1))
        #Capa oculta
        self.Hs[1] = self.tanh(np.dot(self.weigths[1],self.Hs[0]) + self.bias[1])
        #Predicción
        self.Hs[2] = self.soft_max(np.dot(self.weigths[2],self.Hs[1]) + self.bias[2])
        
    def train(self, epochs = 5):
        for epoch in range(epochs):
            for split in range(0,self.split):
                train_split, validation_split =  self.getTrainValidSplits(split)
                count = 0
                for k in train_split:
                    x = self.bigrams[k][0]
                    y = self.bigrams[k][1]
                    self._fordward(x)
                    self._backward(y,x)
                    count += 1 
                    if (count%5000 == 0):
                        print('Epoch:', epoch ,' fold: ',split+1,' iteración: ',count,' / ',len(train_split))    
                #self.validation(validation_split)
                
            
    def validation(self,validation_split):
        sizeValid = len(validation_split)
        prediciones = np.zeros(sizeValid) 
        for k,i in zip(validation_split,range(sizeValid)): #K es muestra
            x = self.bigrams[k][0]
            y = self.bigrams[k][1]
            self._fordward(x)
            prediciones[i] = self.Hs[2][y]
        self.prediciones = prediciones #Solo para debbug
        self.perplejidad.append(np.prod(1/prediciones)**(1/sizeValid))
        print("Perplejidad: ",self.perplejidad[-1],"\n")
        
        
    def predic(self, x):
        self._fordward(x)
        return self.Hs[2]
        
    def loadData(self, bigrams, sizeVocaculary):
        self.bigrams = bigrams
        self.sizeData = len(bigrams)
        self.sizeVocaculary = sizeVocaculary
        self.buildTrainValidationSplits(bigrams, 5)
        
    def buildTrainValidationSplits(self, bigrams, numsplit = 5):
        self.indexes = [i for i in range (len (bigrams))]
        random.shuffle(self.indexes)  
        self.split = numsplit
        self.sizeSplit = len(bigrams)//numsplit
        self.validation_r = [i for i in range(self.split)]
        self.perplejidad = list()
        
    def getTrainValidSplits(self, split):
        ini = (split)*self.sizeSplit
        fin = (split + 1)*self.sizeSplit
        
        if split > 0 and split<self.split-1:
            train_split = self.indexes[0:ini] + self.indexes[fin:]
            validation_split = self.indexes[ini:fin]
        elif split == 0 :
            train_split = self.indexes[fin:]
            validation_split = self.indexes[0:fin]
        else:
            train_split = self.indexes[0:ini]
            validation_split = self.indexes[ini:]
        #Solo para debug
        #self.validation_split = validation_split
        #self.train_split = train_split
        
        return train_split, validation_split
            
    def loadModel(self,file='model.pk'):
        with open(file,'rb') as model_file:
            self.weigths = pickle.load(model_file)
            self.bias = pickle.load(model_file)
        print('¡Modelo cargado correctamente!')
    
    def export(self,file='model_2.pk'):
        with open(file,'wb') as model_file:
            pickle.dump(self.weigths, model_file)
            pickle.dump(self.bias, model_file)
        

In [89]:
with open('Corpus.pk','rb') as corpus_file:
    dic_words = pickle.load(corpus_file)
    dic_index = pickle.load(corpus_file)
    vocabulario = pickle.load(corpus_file)
    bigrams = pickle.load(corpus_file)
    

In [ ]:
model.validation_split

In [129]:
dim = len(vocabulario)
model = ModeloB(dim, [100 , 300], dim, 0.02) #Más alto de 0.1 hace que no converga
model.loadData(bigrams, dim)
model.loadModel('model_13.pk')
model.train(10)
model.export('model_23.pk')

¡Modelo cargado correctamente!
Epoch: 0  fold:  1  iteración:  5000  /  22504
Epoch: 0  fold:  1  iteración:  10000  /  22504
Epoch: 0  fold:  1  iteración:  15000  /  22504
Epoch: 0  fold:  1  iteración:  20000  /  22504
Perplejidad:  inf 

Epoch: 0  fold:  2  iteración:  5000  /  22504
Epoch: 0  fold:  2  iteración:  10000  /  22504
Epoch: 0  fold:  2  iteración:  15000  /  22504
Epoch: 0  fold:  2  iteración:  20000  /  22504
Perplejidad:  inf 

Epoch: 0  fold:  3  iteración:  5000  /  22504
Epoch: 0  fold:  3  iteración:  10000  /  22504
Epoch: 0  fold:  3  iteración:  15000  /  22504
Epoch: 0  fold:  3  iteración:  20000  /  22504
Perplejidad:  inf 

Epoch: 0  fold:  4  iteración:  5000  /  22504
Epoch: 0  fold:  4  iteración:  10000  /  22504
Epoch: 0  fold:  4  iteración:  15000  /  22504
Epoch: 0  fold:  4  iteración:  20000  /  22504
Perplejidad:  inf 

Epoch: 0  fold:  5  iteración:  5000  /  22500
Epoch: 0  fold:  5  iteración:  10000  /  22500
Epoch: 0  fold:  5  iteración:

In [ ]:
model.export('model_3.pk')

In [125]:
import pprint as p
#p.pprint(list(model.prediciones))
p.pprint(list(1/model.prediciones[:5]))
print(np.prod(1/model.prediciones[:5]))
#model._fordward(5)
#model._backward(1,5)

#for m,h in zip(model.weigths,model.Hs):
    #print(m.shape,h.shape)

AttributeError: 'ModeloB' object has no attribute 'prediciones'

In [134]:
p.pprint(list(1/model.prediciones[:10]))
print(np.prod(1/model.prediciones[:10]))

[1.0329704789394227,
 449.133848877946,
 1.0000040964968162,
 45.88753392084974,
 2.775725950143549,
 187.6371349020139,
 2.710478521783928,
 1.027218204273413,
 5.708444824854162,
 1279.7745355590105]
225535817781.74835


In [133]:
palabra = dic_words['pero']
prediccion = model.predic(palabra)
#print(dic_index)
lis = [(dic_index[idx],prob) for idx,prob in zip (range(len(prediccion.T[0])-1),prediccion.T[0])]
sorted(lis, key = lambda x: x[1], reverse=True)[:15]

[('no', 0.13763450033515712),
 ('me', 0.08761853936088525),
 ('es', 0.056657394342152494),
 ('_EOS_', 0.03846127078922387),
 ('que', 0.0339624480540481),
 ('hasta', 0.030048298071716096),
 ('ya', 0.028184453520051396),
 ('y', 0.027752348732223434),
 ('a', 0.0232806070911602),
 ('de', 0.02294203147317707),
 ('si', 0.019754915377732957),
 ('tengo', 0.019483435508565163),
 ('con', 0.018708560823158027),
 ('se', 0.01797349216416634),
 ('en', 0.015888192672174124)]

In [23]:
x = np.array([1,2,3,4,5,6])
print(x[0:2]+x[4:],x[2:4])

np.prod(1/x[:3])**(0.1)

[6 8] [3 4]


0.8359588020779368